In [36]:
import pickle
import os
import sys
import nibabel as nib
import stitchSurfs as stitch
import subprocess
import numpy as np
import projectUtils as prjUtils
import importlib
sys.path.append('/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/')
import gen
import bids_naming as names
importlib.reload(names)

<module 'bids_naming' from '/host/verges/tank/data/daniel/00_commonUtils/00_code/genUtils/bids_naming.py'>

In [ ]:
# equivolumetric surface generation
# options:
# LayNii: Takes cortical ribbon segmentation as input 
#   https://github.com/layerfMRI/LAYNII
# cortPro: takes volumes as input (generates pial, white surfaces)
#   https://github.com/caseypaquola/CortPro
# surface_tools (Kasper Wagstyl): takes pial, white surfaces as input
#   https://github.com/kwagstyl/surface_tools


def runCortPro(study:dict, id:str, ses:str, micro_image_pth:str, dir_surfsIn:str, dir_out:str, n_surfs:int, naming_ptrn:str):
    
    prjUtils.make_dir(dir_out)
    
    fs_dir = "/data/mica1/01_programs/freesurfer-7.3.2" # Path to the FreeSurfer directory [required] (should contain standard license file, 'license.txt')"
    sing_dir  = "/data/mica1/01_programs/micapipe-v0.2.0" # Path to the directory with singularities [required] (must contain micapipe-v0.2.3.simg and, if Freesurfer output is not yet available, fastsurfer_gpu.sif)"
    cortPro_pth = "/host/verges/tank/data/daniel/software/CortPro-main/"

    fs_subjDir = names.get_fsPath(study) # subjects-dir: Path to Freesurfer-style SUBJECTS_DIR [required] (if the directory doesn't contain surfaces for the specified subject, Fastsurfer will be run)
    
    cmd = f"{os.path.join(cortPro_pth, 'microstructure_profiling.sh')} --subject-id {gen.fmt_id_ses(id, ses)}" \
    f" --subjects-dir {fs_subjDir}" \
    f" --micro-image {micro_image_pth}" \
    f" --output-dir {dir_out}" \
    f" --fs-dir {fs_dir}" \
    f" --sing-dir {sing_dir}" \
    f" --num-surfaces {n_surfs}" \
    f" --surface-output {naming_ptrn}" \
    f" --keep-inter-files" # optional

    subprocess.run(cmd, shell=True, capture_output=True, text=True)
    """
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"Error running command: {cmd}")
        print(f"Return code: {result.returncode}")
        print(f"STDOUT: {result.stdout}")
        print(f"STDERR: {result.stderr}")
    else:
        print(f"Command executed successfully: {cmd}")
        if result.stdout:
            print(f"STDOUT: {result.stdout}")
    """
    return


def equiVolSurfs_LayNII(pial, white, n_surfs, out_prefix, laynii_pth = "/host/verges/tank/data/daniel/software/LayNii_v2.9.0"):
    # LayNii: https://github.com/layerfMRI/LAYNII
    # Citation: Huber, L., Poser, B. A., Bandettini, P. A., Arora, K., Wagstyl, K., Cho, S., Goense, J., Nothnagel, N., Morgan, A. T., van den Hurk, J., Mueller A. K., Reynolds, R. C., Glen, D. R., Goebel, R. W., Gulban, O. F. (2021). LayNii: A software suite for layer-fMRI. NeuroImage, 118091. https://doi.org/10.1016/j.neuroimage.2021.118091
    cmd = ""
    subprocess.run(cmd, shell=True)
    return save_names


def mp_MPC(id:str, ses:str, out_pth:str, bids_dir:str, microStrucImg_pth:str, ):
    
    cmd = "micapipe " \
    f"-sub {gen.fmt_id_ses(id, ses)} " \
    f"-out {out_pth} " \
    f"-bids {bids_dir} " \
    f"-MPC -microstructural_img {microStrucImg_pth}"
    subprocess.run(cmd, shell=True)
    return

def run_surfTools(gray, white, n_surfs, out_prefix, smoothing=None, fs:bool=False):
    # https://github.com/kwagstyl/surface_tools
    # uses different versions of python tools, thus run in own conda env
    
    cmd = "conda run -n legacy_kwagstylSurfTools " \
    "generate_equivolumetric_surfaces " \
        f"{gray} {white} {n_surfs} {out_prefix}"
    if fs:
        cmd += f" --software freesurfer"
    if smoothing is not None:
        cmd += f" --smoothing {smoothing}"
    print(cmd)
    return
    result = subprocess.run(cmd, shell=True)
    if result.returncode != 0:
        print(f"Error running command: {cmd}")
        print(f"Return code: {result.returncode}")
        print(f"STDOUT: {result.stdout}")
        print(f"STDERR: {result.stderr}")
    else:
        print(f"Success: {cmd}")
        if result.stdout:
            print(f"STDOUT: {result.stdout}")
    return

PNI = {
    'studyName': 'PNI',
    'studyDescrip': '7T',
    'dir_root': '/data/mica3/BIDS_PNI/',
    'dir_deriv': 'derivatives/',
    'dir_fs': 'fastsurfer/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}

MICS = {
    'studyName': 'MICs',
    'studyDescrip': '3T',
    'dir_root': '/data/mica3/BIDS_MICs/',
    'dir_deriv': 'derivatives/',
    'dir_fs': 'freesurfer/',
    'dir_mp': 'micapipe_v0.2.0/',
    'dir_hu': 'hippunfold_v1.3.0/hippunfold/', # update to v2?
}



In [40]:
# works when specify fastsurfer SUBJECTS_DIR
# adds id_ses file to output dir
# copies input volumes to output dir
runCortPro(study = PNI, id="PNC021", ses="01", 
           micro_image_pth = names.get_volPath(study = PNI, id = "PNC021", ses = "01", volName = "T1map")[0],
           dir_surfsIn="/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI", 
           dir_out="/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/surfs/", 
           n_surfs=5, naming_ptrn="stitched")

KeyboardInterrupt: 

In [27]:
# Create ribbon from surfaces in 'rim' format for LayNII input
# LayNII's LN_RIMIFY does not appear to exist, perhaps under another name
# Note. Must return to volume space then recreate surfaces -- lose vertex correspondence?


root = "/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/"
out_dir = "/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/"
inner=os.path.join(root, "sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii")
outer=os.path.join(root,"sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched.surf.gii")

ref_vol = names.get_volPath(study = PNI, id = "PNC021", ses = "01", volName = "T1map")[0]
ref_img = nib.load(ref_vol)

zeros_data = np.zeros(ref_img.shape, dtype=ref_img.get_fdata().dtype)
ribbon_img = nib.Nifti1Image(zeros_data, affine=ref_img.affine, header=ref_img.header)
nib.save(ribbon_img, os.path.join(out_dir, 'ribbon.nii.gz'))

# 2. Project distance fields from surfaces (use your anatomical ref)
subprocess.run(f"wb_command -surface-closest-vertex {outer} {ref_vol} {os.path.join(out_dir, 'outer_dist.nii.gz')}", shell=True)
subprocess.run(f"wb_command -surface-closest-vertex {inner} {ref_vol} {os.path.join(out_dir, 'inner_dist.nii.gz')}", shell=True)

# 3. Create binary ribbon (outer MINUS inner)
subprocess.run(f"fslmaths {os.path.join(out_dir, 'outer_dist.nii.gz')} -abs -bin {os.path.join(out_dir, 'outer_mask.nii.gz')}", shell=True)
subprocess.run(f"fslmaths {os.path.join(out_dir, 'inner_dist.nii.gz')} -abs -bin {os.path.join(out_dir, 'inner_mask.nii.gz')}", shell=True)
subprocess.run(f"fslmaths {os.path.join(out_dir, 'outer_mask.nii.gz')} -sub {os.path.join(out_dir, 'inner_mask.nii.gz')} {os.path.join(out_dir, 'ribbon.nii.gz')}", shell=True)
subprocess.run(f"fslmaths {os.path.join(out_dir, 'ribbon.nii.gz')} -bin {os.path.join(out_dir, 'ribbon.nii.gz')}", shell=True)

subprocess.run(f"fslmaths {os.path.join(out_dir, 'inner_mask.nii.gz')} -mul 1 {os.path.join(out_dir, 'inner_rim.nii.gz')}", shell=True)
subprocess.run(f"fslmaths {os.path.join(out_dir, 'outer_mask.nii.gz')} -mul 2 {os.path.join(out_dir, 'outer_rim.nii.gz')}", shell=True)
subprocess.run(f"fslmaths {os.path.join(out_dir, 'ribbon.nii.gz')} -mul 3 {os.path.join(out_dir, 'gm_rim.nii.gz')}", shell=True)
subprocess.run(f"fslmaths {os.path.join(out_dir, 'inner_rim.nii.gz')} -add {os.path.join(out_dir, 'outer_rim.nii.gz')} -add {os.path.join(out_dir, 'gm_rim.nii.gz')} {os.path.join(out_dir, 'rim.nii.gz')}", shell=True)




While running:
wb_command -surface-closest-vertex /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-white_hippSurf-den-0p5mm_hippLbl-outer_stitched.surf.gii /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC021/ses-01/maps/sub-PNC021_ses-01_space-nativepro_map-T1map.nii.gz /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/outer_dist.nii.gz

ERROR: did not find any coordinates in file, make sure you use only whitespace to separate numbers


While running:
wb_command -surface-closest-vertex /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC021/ses-01/maps/sub-PNC021_ses-01_space-nativepro_map-T1map.nii.gz /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/inner_dist.nii.gz


CompletedProcess(args='wb_command -surface-closest-vertex /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/sub-PNC021_ses-01_hemi_L_ctxSurf-fsLR-32k_ctxLbl-pial_hippSurf-den-0p5mm_hippLbl-inner_stitched.surf.gii /data/mica3/BIDS_PNI/derivatives/micapipe_v0.2.0/sub-PNC021/ses-01/maps/sub-PNC021_ses-01_space-nativepro_map-T1map.nii.gz /host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/inner_dist.nii.gz', returncode=255)

In [3]:
prjUtils.make_dir("/host/verges/tank/data/daniel/04_inVivoHistology/data/PNI/sub-PNC021_ses-01/surfs/")